In [1]:
import pandas as pd

df1 = pd.read_csv("data/train_dataset.csv")
df2 = pd.read_csv("data/valid_dataset.csv")
df3 = pd.read_csv("data/test_dataset.csv")

df = pd.concat([df1, df2, df3]).reset_index(drop=True)
df = df.loc[:, "Target Sequence"].drop_duplicates().reset_index(drop=True)
df

0       MLKFKYGARNPLDAGAAEPIASRASRLNLFFQGKPPFMTQQQMSPL...
1       MRHSKRTYCPDWDDKDWDYGKWRSSSSHKRRKRSHSSAQENKRCKY...
2       MRHSKRTHCPDWDSRESWGHESYRGSHKRKRRSHSSTQENRHCKPH...
3       MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQ...
4       MVVFNGLLKIKICEAVSLKPTAWSLRHAVGPRPQTFLLDPYIALNV...
                              ...                        
3062    MGEGGAAAALVAAAAAAAAAAAAVVAGQRRRRLGRRARCHGPGRAA...
3063    MAAAAAAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNLNK...
3064    MDRLDANVSSNEGFGSVEKVVLLTFFAMVILMAILGNLLVMVAVCR...
3065    MATALMAVVLRAAAVAPRLRGRGGTGGARRLSCGARRRAARGTSPG...
3066    MAWALLLLTLLTQGTGSWAQSALTQPPSASGSPGQSVTISCTGTSS...
Name: Target Sequence, Length: 3067, dtype: object

In [2]:
import pickle
import pandas as pd
import transformers
from tqdm import tqdm
from transformers import AutoModel, BertTokenizer, RobertaTokenizer

prot_tokenizer = BertTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D", do_lower_case=False)
prot_encoder = AutoModel.from_pretrained("facebook/esm2_t33_650M_UR50D").to("cuda")
prot_encoder.eval()

/home/ming/miniconda3/envs/taohong/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'EsmTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

/home/ming/miniconda3/envs/taohong/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/esm2_t33_650M_UR50D were not used when initializing EsmModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'esm.contact_head.regression.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'esm.contact_head.regression.weight']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to

EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 1280, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1026, 1280, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-32): 33 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=1280, out_features=1280, bias=True)
            (key): Linear(in_features=1280, out_features=1280, bias=True)
            (value): Linear(in_features=1280, out_features=1280, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=1280, out_features=1280, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmInter

In [3]:
import os
os.makedirs('prot_feat', exist_ok=True)

for max_length in [545]:
    results = {}

    for data in tqdm(df, total=len(df)):
        seq = prot_tokenizer(" ".join(data), max_length=max_length + 2, truncation=True, return_tensors="pt").to("cuda")
        a = prot_encoder(**seq)
        a = a.last_hidden_state.detach().to("cpu")
        results[data[:20]] = a[:, 0]

    with open(f"prot_feat/{max_length}_cls.pkl", "wb") as f:
        pickle.dump(results, f)

  0%|                                                  | 0/3067 [00:00<?, ?it/s]


TypeError: forward() got an unexpected keyword argument 'token_type_ids'